In [2]:
import numpy as np
import pandas as pd
import json
import os

In [3]:
def select_run_folder(base_dir, run_number):
    selected_folder_path = os.path.join(base_dir, str(run_number))
    if os.path.isdir(selected_folder_path):
        return selected_folder_path
    else:
        print("No run found for provided number")

In [4]:
base_directory = "./Data_collect"

## Execution Data loading functions

In [25]:
def load_gpu_info_parameters(folder_path, filename='gpu_info.json'):
    json_file_path = os.path.join(folder_path, filename)

    # Load the run parameters from the JSON file
    with open(json_file_path, 'r') as json_file:
        gpu_parameters = json.load(json_file)
        return gpu_parameters
    
def load_profiling_parameters(folder_path,N,theta,dt):
    profiling_filename = f"profiling_{N}_{theta}_{dt}.json"
    json_file_path = os.path.join(folder_path,profiling_filename)
    
    with open(json_file_path, 'r') as json_file:
        profile_parameters = json.load(json_file)
        # Extract units and data
        units = profile_parameters['units']
        data = profile_parameters['data']

        # Convert data back to a DataFrame
        df = pd.DataFrame(data)
        
        return df

    # profile_paths = sorted([os.path.join(json_file_path, file) for file in os.listdir(json_file_path) 
    #                 if file not in ['runs.json','gpu_info.json']], reverse=False)

    # for profile_path in profile_paths:
    #     with open(profile_path, 'r') as json_file:
    #         profile_parameters = json.load(json_file)
    #         # Extract units and data
    #         units = profile_parameters['units']
    #         data = profile_parameters['data']

    #         # Convert data back to a DataFrame
    #         df = pd.DataFrame(data)
            
    #         return df

            # # Insert units as the first row
            # df = pd.concat([pd.DataFrame([units]), df], ignore_index=True)

            # Display the resulting DataFrame

            # Initialize an empty dictionary to collect renamed columns for each row
            
def load_profiling_parameters_ncu(folder_path,N,theta,dt):
    profiling_filename = f"profiling_{N}_{theta}_{dt}.json"
    json_file_path = os.path.join(folder_path,profiling_filename)
    
    with open(json_file_path, 'r') as json_file:
            profile_parameters = json.load(json_file)
            # Extract units and data
            units = profile_parameters['units']
            data = profile_parameters['data']

            # Convert data back to a DataFrame
            df = pd.DataFrame(data)
            df['Metric Value'] = pd.to_numeric(df['Metric Value'].str.replace(',', ''), errors='coerce')

            # Define columns to group by
            group_columns = ['ID', 'Process ID', 'Process Name', 'Host Name', 'Kernel Name', 
                            'Context', 'Stream', 'Block Size', 'Grid Size', 'Device', 'CC', 'Section Name']

            # Pivot the DataFrame to aggregate by the specified columns, with Metric Name as new columns
            df_pivot = df.pivot_table(
                index=group_columns,
                columns='Metric Name',
                values='Metric Value',
                aggfunc='first'  # Use 'first' since each group should have unique Metric Values
            ).reset_index()

            # Flatten multi-level columns if necessary (e.g., after pivot)
            df_pivot.columns.name = None  # Remove the column group name
            df_pivot.columns = [col if not isinstance(col, tuple) else col[1] for col in df_pivot.columns]
            
            return df_pivot

def base_filter(prof_df):    
    filter_out = [
        "Device",
        "Context",
        "Stream",
        "Correlation_ID",
        "tex0_cache_sector_queries",
        "tex1_cache_sector_queries",
        "tex0_cache_sector_misses",
        "tex1_cache_sector_misses",
        "l2_subp0_write_sector_misses",
        "l2_subp1_write_sector_misses",
        "l2_subp0_read_sysmem_sector_queries",
        "l2_subp1_read_sysmem_sector_queries",
        "gld_inst_8bit",
        "gld_inst_16bit",
        "gld_inst_32bit",
        "gld_inst_64bit",
        "gld_inst_128bit",
        "gst_inst_8bit",
        "gst_inst_16bit",
        "gst_inst_32bit",
        "gst_inst_64bit",
        "gst_inst_128bit",
        "prof_trigger_00",
        "prof_trigger_01",
        "prof_trigger_02",
        "prof_trigger_03",
        "prof_trigger_04",
        "prof_trigger_05",
        "prof_trigger_06",
        "prof_trigger_07",
        "inst_issued2",
        "generic_load",
        "generic_store",
        "inst_executed_in_trap",
        "active_cycles_in_trap",
        "inst_executed_shared_atom_cas",
        "shared_atom",
        "global_atom_cas",
        "gred_count",
        "divergent_branch",
        "branch",
        "shared_ld_bank_conflict",
        "shared_st_bank_conflict",
        "inst_replay_overhead",
        "shared_load_transactions_per_request",
        "shared_store_transactions_per_request",
        "local_hit_rate",
        "gld_requested_throughput",
        "gst_requested_throughput",
        "local_memory_overhead",
        "tex_cache_throughput",
        "sysmem_utilization",
        "stall_sync",
        "stall_texture",
        "shared_efficiency",
        "cf_issued",
        "ldst_issued",
        "ldst_executed",
        "atomic_transactions_per_request",
        "flop_count_hp",
        "flop_count_hp_add",
        "flop_count_hp_mul",
        "flop_count_hp_fma",
        "inst_fp_16",
        "sysmem_read_utilization",
        "sysmem_write_utilization",
        "pcie_total_data_transmitted",
        "pcie_total_data_received",
        "inst_executed_surface_loads",
        "inst_executed_surface_stores",
        "inst_executed_global_atomics",
        "inst_executed_global_reductions",
        "inst_executed_surface_atomics",
        "inst_executed_surface_reductions",
        "inst_executed_shared_atomics",
        "inst_executed_tex_ops",
        "l2_surface_load_bytes",
        "l2_global_reduction_bytes",
        "l2_surface_store_bytes",
        "l2_surface_reduction_bytes",
        "l2_surface_atomic_store_bytes",
        "surface_load_requests",
        "surface_store_requests",
        "global_reduction_requests",
        "surface_atomic_requests",
        "surface_reduction_requests",
        "sysmem_read_bytes",
        "sysmem_write_bytes",
        "texture_load_requests",
        "unique_warps_launched",
        "tex_utilization",
        "l2_utilization",
        "shared_utilization",
        "ldst_fu_utilization",
        "cf_fu_utilization",
        "special_fu_utilization",
        "tex_fu_utilization",
        "single_precision_fu_utilization",
        "double_precision_fu_utilization",
        "dram_utilization",
        "half_precision_fu_utilization",
        "ecc_transactions",
        "ecc_throughput",
        "l2_global_load_bytes",
        "l2_local_load_bytes",
        "l2_surface_load_bytes",
        "l2_local_global_store_bytes",
        "l2_global_reduction_bytes",
        "l2_global_atomic_store_bytes",
        "l2_surface_store_bytes",
        "l2_surface_reduction_bytes",
        "l2_surface_atomic_store_bytes"
        ]
    return prof_df.drop(filter_out, axis=1, errors='ignore')

def ncu_2_nvprof(df):
    nv = {}
    nv["Device"] = "NVIDIA GeForce RTX 3070 8GB (0)"
    nv["Context"] = df['Context']
    nv["Stream"] = df['Stream']
    nv["Kernel"] = df['Kernel Name']
    nv["Correlation_ID"] = ""
    nv["tex0_cache_sector_queries"] = ""
    nv["tex1_cache_sector_queries"] = ""
    nv["tex0_cache_sector_misses"] = ""
    nv["tex1_cache_sector_misses"] = ""
    nv["fb_subp0_read_sectors"] = df['dram__sectors_read.sum']
    nv["fb_subp1_read_sectors"] = df['dram__sectors_read.sum']
    nv['fb_subp0_write_sectors'] = df['dram__sectors_write.sum']
    nv['fb_subp1_write_sectors'] = df['dram__sectors_write.sum']
    nv['l2_subp0_write_sector_misses'] = df['lts__t_sectors_op_write_lookup_miss.sum']
    nv['l2_subp1_write_sector_misses'] = df['lts__t_sectors_op_write_lookup_miss.sum']
    nv['l2_subp0_read_sector_misses'] = df['lts__t_sectors_op_read_lookup_miss.sum']
    nv['l2_subp1_read_sector_misses'] = df['lts__t_sectors_op_read_lookup_miss.sum']
    nv['l2_subp0_read_tex_sector_queries'] = df['lts__t_sectors_srcunit_tex_op_read.sum']
    nv['l2_subp1_read_tex_sector_queries'] = df['lts__t_sectors_srcunit_tex_op_read.sum']
    nv['l2_subp0_write_tex_sector_queries'] = df['lts__t_sectors_srcunit_tex_op_write.sum']
    nv['l2_subp1_write_tex_sector_queries'] = df['lts__t_sectors_srcunit_tex_op_write.sum']
    nv['l2_subp0_read_tex_hit_sectors'] = df['lts__t_sectors_srcunit_tex_op_read_lookup_hit.sum']
    nv['l2_subp1_read_tex_hit_sectors'] = df['lts__t_sectors_srcunit_tex_op_read_lookup_hit.sum']
    nv['l2_subp0_write_tex_hit_sectors'] = df['lts__t_sectors_srcunit_tex_op_write_lookup_hit.sum']
    nv['l2_subp1_write_tex_hit_sectors'] = df['lts__t_sectors_srcunit_tex_op_write_lookup_hit.sum']
    nv['l2_subp0_total_read_sector_queries'] = df['lts__t_sectors_op_read.sum'] + df['lts__t_sectors_op_atom.sum'] + df['lts__t_sectors_op_red.sum']
    nv['l2_subp1_total_read_sector_queries'] = df['lts__t_sectors_op_read.sum'] + df['lts__t_sectors_op_atom.sum'] + df['lts__t_sectors_op_red.sum']
    nv['l2_subp0_total_write_sector_queries'] = df['lts__t_sectors_op_write.sum'] + df['lts__t_sectors_op_atom.sum'] + df['lts__t_sectors_op_red.sum']
    nv['l2_subp1_total_write_sector_queries'] = df['lts__t_sectors_op_write.sum'] + df['lts__t_sectors_op_atom.sum'] + df['lts__t_sectors_op_red.sum']
    nv['l2_subp0_read_sysmem_sector_queries'] = df['lts__t_sectors_aperture_sysmem_op_read.sum']
    nv['l2_subp1_read_sysmem_sector_queries'] = df['lts__t_sectors_aperture_sysmem_op_read.sum']
    nv['l2_subp0_write_sysmem_sector_queries'] = df['lts__t_sectors_aperture_sysmem_op_write.sum']
    nv['l2_subp1_write_sysmem_sector_queries'] = df['lts__t_sectors_aperture_sysmem_op_write.sum']
    nv['elapsed_cycles_sm'] = df['sm__cycles_elapsed.sum']
    nv['gld_inst_8bit'] = ''
    nv['gld_inst_16bit'] = ''
    nv['gld_inst_32bit'] = ''
    nv['gld_inst_64bit'] = ''
    nv['gld_inst_128bit'] = ''
    nv['gst_inst_8bit'] = ''
    nv['gst_inst_16bit'] = ''
    nv['gst_inst_32bit'] = ''
    nv['gst_inst_64bit'] = ''
    nv['gst_inst_128bit'] = ''
    nv['prof_trigger_00'] = ''
    nv['prof_trigger_01'] = ''
    nv['prof_trigger_02'] = ''
    nv['prof_trigger_03'] = ''
    nv['prof_trigger_04'] = ''
    nv['prof_trigger_05'] = ''
    nv['prof_trigger_06'] = ''
    nv['prof_trigger_07'] = ''
    nv['warps_launched'] = df['smsp__warps_launched.sum']
    nv['inst_issued0'] = df['smsp__issue_inst0.sum']
    nv['inst_issued1'] = df['sm__inst_issued.sum']
    nv['inst_issued2'] = ''
    nv['inst_executed'] = df['sm__inst_executed.sum']
    nv['thread_inst_executed'] = df['smsp__thread_inst_executed.sum']
    nv['not_predicated_off_thread_inst_executed'] = df['smsp__thread_inst_executed_pred_on.sum']
    nv['active_cycles'] = df['sm__cycles_active.sum']
    nv['active_warps'] = df['sm__warps_active.sum']
    nv['global_load'] = df['smsp__inst_executed_op_global_ld.sum']
    nv['global_store'] = df['smsp__inst_executed_op_global_st.sum']
    nv['generic_load'] = df['smsp__inst_executed_op_generic_ld.sum']
    nv['generic_store'] = df['smsp__inst_executed_op_generic_st.sum']
    nv['inst_executed_in_trap'] = ''
    nv['active_cycles_in_trap'] = ''
    nv['local_store'] = df['smsp__inst_executed_op_local_st.sum']
    nv['local_load'] = df['smsp__inst_executed_op_local_ld.sum']
    nv['shared_load'] = df['smsp__inst_executed_op_shared_ld.sum']
    nv['shared_store'] = df['smsp__inst_executed_op_shared_st.sum']
    nv['inst_executed_shared_atom_cas'] = ''
    nv['shared_atom'] = df['smsp__inst_executed_op_shared_atom.sum']
    nv['global_atom_cas'] = df['smsp__inst_executed_op_generic_atom_dot_cas.sum']
    nv['atom_count'] = df['smsp__inst_executed_op_generic_atom_dot_alu.sum']
    nv['gred_count'] = df['smsp__inst_executed_op_global_red.sum']
    nv['divergent_branch'] = ''
    nv['branch'] = ''
    nv['shared_ld_bank_conflict'] = df['l1tex__data_bank_conflicts_pipe_lsu_mem_shared_op_ld.sum']
    nv['shared_st_bank_conflict'] = df['l1tex__data_bank_conflicts_pipe_lsu_mem_shared_op_st.sum']
    nv['shared_ld_transactions'] = df['l1tex__data_pipe_lsu_wavefronts_mem_shared_op_ld.sum']
    nv['shared_st_transactions'] = df['l1tex__data_pipe_lsu_wavefronts_mem_shared_op_st.sum']
    nv['active_cycles_pm'] = df['sm__cycles_active.sum']
    nv['active_warps_pm'] = df['sm__warps_active.sum']
    nv['elapsed_cycles_pm'] = df['sm__cycles_elapsed.sum']
    nv['inst_per_warp'] = df['smsp__average_inst_executed_per_warp.ratio']
    nv['branch_efficiency'] = df['smsp__sass_average_branch_targets_threads_uniform.pct']
    nv['warp_execution_efficiency'] = df['smsp__thread_inst_executed_per_inst_executed.ratio']
    nv['warp_nonpred_execution_efficiency'] = df['smsp__thread_inst_executed_per_inst_executed.pct']
    nv['inst_replay_overhead'] = ''
    nv['shared_load_transactions_per_request'] = ''
    nv['shared_store_transactions_per_request'] = ''
    nv['local_load_transactions_per_request'] = df['l1tex__average_t_sectors_per_request_pipe_lsu_mem_local_op_ld.ratio']
    nv['local_store_transactions_per_request'] = df['l1tex__average_t_sectors_per_request_pipe_lsu_mem_local_op_st.ratio']
    nv['gld_transactions_per_request'] = df['l1tex__average_t_sectors_per_request_pipe_lsu_mem_global_op_ld.ratio']
    nv['gst_transactions_per_request'] = df['l1tex__average_t_sectors_per_request_pipe_lsu_mem_global_op_st.ratio']
    nv['shared_store_transactions'] = df['l1tex__data_pipe_lsu_wavefronts_mem_shared_op_st.sum']
    nv['shared_load_transactions'] = df['l1tex__data_pipe_lsu_wavefronts_mem_shared_op_ld.sum']
    nv['local_load_transactions'] = df['l1tex__t_sectors_pipe_lsu_mem_local_op_ld.sum']
    nv['local_store_transactions'] = df['l1tex__t_sectors_pipe_lsu_mem_local_op_st.sum']
    nv['gld_transactions'] = df['l1tex__t_sectors_pipe_lsu_mem_global_op_ld.sum']
    nv['gst_transactions'] = df['l1tex__t_sectors_pipe_lsu_mem_global_op_st.sum']
    nv['sysmem_read_transactions'] = df['lts__t_sectors_aperture_sysmem_op_read.sum']
    nv['sysmem_write_transactions'] = df['lts__t_sectors_aperture_sysmem_op_write.sum']
    nv['l2_read_transactions'] = df['lts__t_sectors_op_read.sum'] + df['lts__t_sectors_op_atom.sum'] + df['lts__t_sectors_op_red.sum']
    nv['l2_write_transactions'] = df['lts__t_sectors_op_write.sum'] + df['lts__t_sectors_op_atom.sum'] + df['lts__t_sectors_op_red.sum']
    nv['global_hit_rate'] = (df['l1tex__t_sectors_pipe_lsu_mem_global_op_ld_lookup_hit.sum'] + df['l1tex__t_sectors_pipe_lsu_mem_global_op_st_lookup_hit.sum'] + df['l1tex__t_sectors_pipe_lsu_mem_global_op_red_lookup_hit.sum'] + df['l1tex__t_sectors_pipe_lsu_mem_global_op_atom_lookup_hit.sum']) / (df['l1tex__t_sectors_pipe_lsu_mem_global_op_ld.sum'] + df['l1tex__t_sectors_pipe_lsu_mem_global_op_st.sum'] + df['l1tex__t_sectors_pipe_lsu_mem_global_op_red.sum'] + df['l1tex__t_sectors_pipe_lsu_mem_global_op_atom.sum'])
    nv['local_hit_rate'] = ''
    nv['gld_requested_throughput'] = ''
    nv['gst_requested_throughput'] = ''
    nv['gld_throughput'] = df['l1tex__t_bytes_pipe_lsu_mem_global_op_ld.sum.per_second']
    nv['gst_throughput'] = df['l1tex__t_bytes_pipe_lsu_mem_global_op_st.sum.per_second']
    nv['local_memory_overhead'] = ''
    nv['tex_cache_hit_rate'] = df['l1tex__t_sector_hit_rate.pct']
    nv['l2_tex_read_hit_rate'] = df['lts__t_sector_op_read_hit_rate.pct']
    nv['l2_tex_write_hit_rate'] = df['lts__t_sector_op_write_hit_rate.pct']
    nv['tex_cache_throughput'] = ''
    nv['l2_tex_read_throughput'] = df['lts__t_sectors_srcunit_tex_op_read.sum.per_second']
    nv['l2_tex_write_throughput'] = df['lts__t_sectors_srcunit_tex_op_write.sum.per_second']
    nv['l2_read_throughput'] = df['lts__t_sectors_op_read.sum.per_second'] + df['lts__t_sectors_op_atom.sum.per_second'] + df['lts__t_sectors_op_red.sum.per_second']
    nv['l2_write_throughput'] = df['lts__t_sectors_op_write.sum.per_second'] + df['lts__t_sectors_op_atom.sum.per_second'] + df['lts__t_sectors_op_red.sum.per_second']
    nv['sysmem_read_throughput'] = df['lts__t_sectors_aperture_sysmem_op_read.sum.per_second']
    nv['sysmem_write_throughput'] = df['lts__t_sectors_aperture_sysmem_op_write.sum.per_second']
    nv['local_load_throughput'] = df['l1tex__t_bytes_pipe_lsu_mem_local_op_ld.sum.per_second']
    nv['local_store_throughput'] = df['l1tex__t_sectors_pipe_lsu_mem_local_op_st.sum.per_second']
    nv['shared_load_throughput'] = df['l1tex__data_pipe_lsu_wavefronts_mem_shared_op_ld.sum.per_second']
    nv['shared_store_throughput'] = df['l1tex__data_pipe_lsu_wavefronts_mem_shared_op_st.sum.per_second']
    nv['gld_efficiency'] = df['smsp__sass_average_data_bytes_per_sector_mem_global_op_ld.pct']
    nv['gst_efficiency'] = df['smsp__sass_average_data_bytes_per_sector_mem_global_op_st.pct']
    nv['tex_cache_transactions'] = df['l1tex__lsu_writeback_active.avg.pct_of_peak_sustained_active'] + df['l1tex__tex_writeback_active.avg.pct_of_peak_sustained_active']
    nv['flop_count_dp'] = df['smsp__sass_thread_inst_executed_op_dadd_pred_on.sum'] + df['smsp__sass_thread_inst_executed_op_dmul_pred_on.sum'] + df['smsp__sass_thread_inst_executed_op_dfma_pred_on.sum'] * 2
    nv['flop_count_dp_add'] = df['smsp__sass_thread_inst_executed_op_dadd_pred_on.sum']
    nv['flop_count_dp_fma'] = df['smsp__sass_thread_inst_executed_op_dfma_pred_on.sum']
    nv['flop_count_dp_mul'] = df['smsp__sass_thread_inst_executed_op_dmul_pred_on.sum']
    nv['inst_issued'] = df['smsp__inst_issued.sum']
    nv['sysmem_utilization'] = ''
    nv['stall_inst_fetch'] = df['smsp__warp_issue_stalled_no_instruction_per_warp_active.pct']
    nv['stall_exec_dependency'] = df['smsp__warp_issue_stalled_short_scoreboard_per_warp_active.pct'] + df['smsp__warp_issue_stalled_wait_per_warp_active.pct'] 
    nv['stall_memory_dependency'] = df['smsp__warp_issue_stalled_long_scoreboard_per_warp_active.pct']
    nv['stall_texture'] = df['smsp__warp_issue_stalled_tex_throttle_per_warp_active.pct']
    nv['stall_sync'] = df['smsp__warp_issue_stalled_barrier_per_warp_active.pct'] + df['smsp__warp_issue_stalled_membar_per_warp_active.pct']
    nv['stall_other'] = df['smsp__warp_issue_stalled_dispatch_stall_per_warp_active.pct'] + df['smsp__warp_issue_stalled_misc_per_warp_active.pct']
    nv['stall_constant_memory_dependency'] = df['smsp__warp_issue_stalled_imc_miss_per_warp_active.pct']
    nv['stall_pipe_busy'] = df['smsp__warp_issue_stalled_math_pipe_throttle_per_warp_active.pct'] + df['smsp__warp_issue_stalled_mio_throttle_per_warp_active.pct']
    nv['shared_efficiency'] = ''
    nv['inst_fp_64'] = df['smsp__sass_thread_inst_executed_op_fp64_pred_on.sum']
    nv['inst_bit_convert'] = df['smsp__sass_thread_inst_executed_op_conversion_pred_on.sum']
    nv['inst_control'] = df['smsp__sass_thread_inst_executed_op_control_pred_on.sum']
    nv['inst_compute_ld_st'] = df['smsp__sass_thread_inst_executed_op_memory_pred_on.sum']
    nv['inst_misc'] = df['smsp__sass_thread_inst_executed_op_misc_pred_on.sum']
    nv['issue_slots'] = df['smsp__inst_issued.sum']
    nv['cf_issued'] = ''
    nv['cf_executed'] = df['smsp__inst_executed_pipe_cbu.sum'] + df['smsp__inst_executed_pipe_adu.sum']
    nv['ldst_issued'] = ''
    nv['ldst_executed'] = ''
    nv['atomic_transactions'] = df['l1tex__t_set_accesses_pipe_lsu_mem_global_op_atom.sum']  + df['l1tex__t_set_accesses_pipe_lsu_mem_global_op_red.sum']
    nv['atomic_transactions_per_request'] = (df['l1tex__t_sectors_pipe_lsu_mem_global_op_atom.sum'] + df['l1tex__t_sectors_pipe_lsu_mem_global_op_red.sum']) / (df['l1tex__t_requests_pipe_lsu_mem_global_op_atom.sum'] + df['l1tex__t_requests_pipe_lsu_mem_global_op_red.sum'])
    nv['l2_atomic_throughput'] = 2 * (df['lts__t_sectors_op_atom.sum.per_second'] + df['lts__t_sectors_op_red.sum.per_second'])
    nv['l2_atomic_transactions'] = 2 * (df['lts__t_sectors_op_atom.sum'] + df['lts__t_sectors_op_red.sum'])
    nv['l2_tex_read_transactions'] = df['lts__t_sectors_srcunit_tex_op_read.sum']
    nv['stall_memory_throttle'] = df['smsp__warp_issue_stalled_drain_per_warp_active.pct'] + df['smsp__warp_issue_stalled_lg_throttle_per_warp_active.pct']
    nv['stall_not_selected'] = df['smsp__warp_issue_stalled_not_selected_per_warp_active.pct']
    nv['l2_tex_write_transactions'] = df['lts__t_sectors_srcunit_tex_op_write.sum']
    nv['flop_count_hp'] = df['smsp__sass_thread_inst_executed_op_hadd_pred_on.sum'] + df['smsp__sass_thread_inst_executed_op_hmul_pred_on.sum'] + df['smsp__sass_thread_inst_executed_op_hfma_pred_on.sum'] * 2
    nv['flop_count_hp_add'] = df['smsp__sass_thread_inst_executed_op_hadd_pred_on.sum']
    nv['flop_count_hp_mul'] = df['smsp__sass_thread_inst_executed_op_hmul_pred_on.sum']
    nv['flop_count_hp_fma'] = df['smsp__sass_thread_inst_executed_op_hfma_pred_on.sum']
    nv['inst_fp_16'] = df['smsp__sass_thread_inst_executed_op_fp16_pred_on.sum']
    nv['sysmem_read_utilization'] = ''
    nv['sysmem_write_utilization'] = ''
    nv['pcie_total_data_transmitted'] = ''
    nv['pcie_total_data_received'] = ''
    nv['inst_executed_global_loads'] = df['smsp__inst_executed_op_global_ld.sum']
    nv['inst_executed_local_loads'] = df['smsp__inst_executed_op_local_ld.sum']
    nv['inst_executed_shared_loads'] = df['smsp__inst_executed_op_shared_ld.sum']
    nv['inst_executed_surface_loads'] = df['smsp__inst_executed_op_surface_ld.sum'] + df['smsp__inst_executed_op_shared_atom_dot_alu.sum'] + df['smsp__inst_executed_op_shared_atom_dot_cas.sum']
    nv['inst_executed_global_stores'] = df['smsp__inst_executed_op_global_st.sum']
    nv['inst_executed_local_stores'] = df['smsp__inst_executed_op_local_st.sum']
    nv['inst_executed_shared_stores'] = df['smsp__inst_executed_op_shared_st.sum']
    nv['inst_executed_surface_stores'] = df['smsp__inst_executed_op_surface_st.sum']
    nv['inst_executed_global_atomics'] = df['smsp__sass_inst_executed_op_global_atom.sum']
    nv['inst_executed_global_reductions'] = df['smsp__inst_executed_op_global_red.sum']
    nv['inst_executed_surface_atomics'] = df['smsp__inst_executed_op_surface_atom.sum']
    nv['inst_executed_surface_reductions'] = df['smsp__inst_executed_op_surface_red.sum']
    nv['inst_executed_shared_atomics'] = df['smsp__inst_executed_op_shared_atom.sum'] + df['smsp__inst_executed_op_shared_atom_dot_alu.sum'] + df['smsp__inst_executed_op_shared_atom_dot_cas.sum']
    nv['inst_executed_tex_ops'] = df['smsp__inst_executed_op_texture.sum']
    nv['l2_global_load_bytes'] = '' #df['lts__t_bytes_equiv_l1sectormiss_pipe_lsu_mem_global_op_ld.sum']
    nv['l2_local_load_bytes'] = '' #df['lts__t_bytes_equiv_l1sectormiss_pipe_lsu_mem_local_op_ld.sum']
    nv['l2_surface_load_bytes'] = '' #df['lts__t_bytes_equiv_l1sectormiss_pipe_tex_mem_surface_op_ld.sum']
    nv['l2_local_global_store_bytes'] = '' #df['lts__t_bytes_equiv_l1sectormiss_pipe_lsu_mem_local_op_st.sum'] + df['lts__t_bytes_equiv_l1sectormiss_pipe_lsu_mem_global_op_st.sum']
    nv['l2_global_reduction_bytes'] = ''
    nv['l2_global_atomic_store_bytes'] = '' #df['lts__t_bytes_equiv_l1sectormiss_pipe_lsu_mem_global_op_atom.sum']
    nv['l2_surface_store_bytes'] = '' #df['lts__t_bytes_equiv_l1sectormiss_pipe_tex_mem_surface_op_st.sum']
    nv['l2_surface_reduction_bytes'] = ''
    nv['l2_surface_atomic_store_bytes'] = ''
    nv['global_load_requests'] = df['l1tex__t_requests_pipe_lsu_mem_global_op_ld.sum']
    nv['local_load_requests'] = df['l1tex__t_requests_pipe_lsu_mem_local_op_ld.sum']
    nv['surface_load_requests'] = df['l1tex__t_requests_pipe_tex_mem_surface_op_ld.sum']
    nv['global_store_requests'] = df['l1tex__t_requests_pipe_lsu_mem_global_op_st.sum']
    nv['local_store_requests'] = df['l1tex__t_requests_pipe_lsu_mem_local_op_st.sum']
    nv['surface_store_requests'] = df['l1tex__t_requests_pipe_tex_mem_surface_op_st.sum']
    nv['global_atomic_requests'] = df['l1tex__t_requests_pipe_lsu_mem_global_op_atom.sum']
    nv['global_reduction_requests'] = df['l1tex__t_requests_pipe_lsu_mem_global_op_red.sum']
    nv['surface_atomic_requests'] = df['l1tex__t_requests_pipe_tex_mem_surface_op_atom.sum']
    nv['surface_reduction_requests'] = df['l1tex__t_requests_pipe_tex_mem_surface_op_red.sum']
    nv['sysmem_read_bytes'] = ''
    nv['sysmem_write_bytes'] = ''
    nv['l2_tex_hit_rate'] = df['lts__t_sector_hit_rate.pct']
    nv['texture_load_requests'] = df['l1tex__t_requests_pipe_tex_mem_texture.sum']
    nv['unique_warps_launched'] = ''
    nv['sm_efficiency'] = df['smsp__cycles_active.avg.pct_of_peak_sustained_elapsed']
    nv['achieved_occupancy'] = df['sm__warps_active.avg.pct_of_peak_sustained_active'] / 100
    nv['ipc'] = df['smsp__inst_executed.avg.per_cycle_active']
    nv['issued_ipc'] = df['smsp__inst_issued.avg.per_cycle_active']
    nv['issue_slot_utilization'] = df['smsp__issue_active.avg.pct_of_peak_sustained_active']
    nv['eligible_warps_per_cycle'] = df['smsp__warps_eligible.sum.per_cycle_active']
    nv['tex_utilization'] = ''
    nv['l2_utilization'] = df['lts__t_sectors.avg.pct_of_peak_sustained_elapsed']
    nv['shared_utilization'] = df['l1tex__data_pipe_lsu_wavefronts_mem_shared.avg.pct_of_peak_sustained_elapsed']
    nv['ldst_fu_utilization'] = df['smsp__inst_executed_pipe_lsu.avg.pct_of_peak_sustained_active']
    nv['cf_fu_utilization'] = ''
    nv['special_fu_utilization'] = df['smsp__inst_executed_pipe_xu.avg.pct_of_peak_sustained_active']
    nv['tex_fu_utilization'] = df['smsp__inst_executed_pipe_tex.avg.pct_of_peak_sustained_active']
    nv['single_precision_fu_utilization'] = df['smsp__pipe_fma_cycles_active.avg.pct_of_peak_sustained_active']
    nv['double_precision_fu_utilization'] = df['smsp__inst_executed_pipe_fp64.avg.pct_of_peak_sustained_active']
    nv['dram_read_transactions'] = df['dram__sectors_read.sum']
    nv['dram_write_transactions'] = df['dram__sectors_write.sum']
    nv['dram_read_throughput'] = df['dram__bytes_read.sum.per_second']
    nv['dram_write_throughput'] = df['dram__bytes_write.sum.per_second']
    nv['dram_utilization'] = df['dram__throughput.avg.pct_of_peak_sustained_elapsed']
    nv['half_precision_fu_utilization'] = ''
    nv['ecc_transactions'] = ''
    nv['ecc_throughput'] = ''
    nv['dram_read_bytes'] = df['dram__bytes_read.sum']
    nv['dram_write_bytes'] = df['dram__bytes_write.sum']

    return pd.DataFrame(nv)

def profile_flatten(df):
    renamed_columns = {}

    # Loop through each row, renaming columns and storing them in the dictionary
    for _, row in df.iterrows():
        kernel = row['Kernel']
        renamed_row = row.drop('Kernel')  # Drop the 'Kernel' column
        renamed_row.index = [f"{col} ({kernel})" for col in renamed_row.index]  # Rename columns with 'Kernel'
        renamed_columns.update(renamed_row.to_dict())  # Add to dictionary

    # Create a DataFrame from the dictionary as a single row
    final_df = pd.DataFrame([renamed_columns])

    return final_df

def load_run_parameters(folder_path, filename='runs.json'):
    json_file_path = os.path.join(folder_path, filename)
    
    # Load the run parameters from the JSON file
    with open(json_file_path, 'r') as json_file:
        run_parameters = json.load(json_file)
        return run_parameters

def load_all_parameters(folder_path, profiler='nvprof'):

    gpu_parameters = load_gpu_info_parameters(folder_path)
    run_parameters = load_run_parameters(folder_path)

    for idx, param in enumerate(run_parameters):
        print(idx)
        param = param | gpu_parameters[0]
    
    # return run_parameters
        if param["profiling"] is True:  
            N = param["N"]
            theta = param["theta"]
            dt = param["dt"]

            if profiler == 'nvprof':
                profiling_df = load_profiling_parameters(folder_path,N,theta,dt)
                
            if profiler == 'ncu':
                profiling_df = load_profiling_parameters_ncu(folder_path,N,theta,dt)
                profiling_df = ncu_2_nvprof(profiling_df)

            profiling_df = base_filter(profiling_df)
            profiling_df = profile_flatten(profiling_df)

        param = param | profiling_df.to_dict(orient="records")[0]
        run_parameters[idx] = param

    return pd.DataFrame(run_parameters)

### Load 1060

In [27]:
current_run_directory = select_run_folder(base_directory,2)
gpu1 = load_all_parameters(current_run_directory,"nvprof")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

### Load 3070

In [28]:
current_run_directory = select_run_folder(base_directory,6)
gpu2 = load_all_parameters(current_run_directory,"ncu")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

### Load 3060

In [29]:
current_run_directory = select_run_folder(base_directory,7)
gpu3 = load_all_parameters(current_run_directory,"ncu")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

### Load 2070 SUPER

In [30]:
current_run_directory = select_run_folder(base_directory,8)
gpu4 = load_all_parameters(current_run_directory,"ncu")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

### Load 4060 Ti

In [31]:
current_run_directory = select_run_folder(base_directory,9)
gpu5 = load_all_parameters(current_run_directory,"ncu")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

### Generate full

In [32]:
full = pd.concat([gpu1,gpu2,gpu3,gpu4,gpu5]).reset_index(drop=True)

In [4]:
full.to_csv("Full/full_3.csv", sep=',')

NameError: name 'full' is not defined

## Load latest full

In [3]:
full = pd.read_csv("Full/full_3.csv", index_col=0)

/tmp/ipykernel_1896116/2210809073.py:1: DtypeWarning: Columns (58,76,78,100,153) have mixed types. Specify dtype option on import or set low_memory=False.
  full = pd.read_csv("Full/full_3.csv", index_col=0)


In [4]:
full = full.drop(['dram_read_bytes (dev_approximate_gravity)'], axis=1, errors='ignore')

## Error data loading functions

In [1]:
def load_error_parameters(folder_path, filename='errors.json'):
    json_file_path = os.path.join(folder_path, filename)
    
    # Load the run parameters from the JSON file
    with open(json_file_path, 'r') as json_file:
        run_parameters = json.load(json_file)
        return run_parameters
    
def process_error_parameters(full_err):
    rows = []
    for entry in full_err:
        N, theta, dt, I_max, errors = entry['N'], entry['theta'], entry['dt'], entry['I'], entry['energy_error_list']
        for I in range(1, len(errors) + 1):
            accumulated_error = sum(errors[:I])  # Cumulative sum up to I
            rows.append({'N': N, 'theta': theta, 'dt': dt, 'I': I, 'accumulated_error': accumulated_error})
    full_err_df = pd.DataFrame(rows)
    return full_err_df

### Generate full errors

In [2]:
base_directory = "Error_collect2"
full_err = load_error_parameters(base_directory)
full_err_df = process_error_parameters(full_err)

full_err_df.to_csv("Full/full_err.csv", sep=',')

NameError: name 'os' is not defined

## Load latest full error

In [3]:
full_err_df = pd.read_csv("Full/full_err.csv", index_col=0)